<a href="https://colab.research.google.com/github/maicon-reis/bandas_de_bollinger/blob/main/fun%C3%A7%C3%A3o_cria_bandas_de_bollinger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
É necessário estar com a biblioteca ```yfinance```, caso não a tenha instalado,
ou esteja utilizando o Google Colaboratory, descomentar a próxima linha antes de
rodar o código
'''
# !pip install -q yfinance
# bibliotecas
import datetime as dt
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.dates import  DateFormatter, MonthLocator, MONDAY

def cria_bb(ticker: str, inicio, fim, p = 20, dp = 2 ) -> pd.core.frame.DataFrame:
    '''
    Esta função recebe o código de uma ação  com os períodos de ínício e fim
    respectivamente, e, caso não seja informado o período e desvio que serão
    utilizados, como padrão utilizará 20 períodos e 2 desvios.
    '''
    # coletando dados
    dados = yf.download( ticker
                        , start=inicio
                        , end=fim )

    # coletando média móvel de p períodos
    dados.loc[ :, 'MM'] = dados[ 'Adj Close' ].rolling( p ).mean()

    # calcular o desvio padrão para uma janela de p períodos
    dados.loc[ :, 'desvio' ] = dados[ 'Adj Close' ].rolling( p ).std()

    # excluir as linhas ausentes
    dados.fillna( value=0, inplace=True )

    # calcular bandas, superior e inferior
    dados.loc[ :, 'Band_Sup' ] = dados[ 'MM' ] + ( dados.desvio * dp )
    dados.loc[ :, 'Band_Inf' ] = dados['MM'] - ( dados.desvio * dp )
    return dados

def cria_grafico(df):
    '''
    Esta função cria um gráfico de linhas utilizando um DataFrame que contenha,
    pelo menos, as seguintes colunas: Adj Close, Ban_Sup, Band_Inf e MM.
    '''
    # configurações iniciais
    months = MonthLocator(range(1,13), bymonthday=1, interval=1)
    monthsFmt = DateFormatter("%b %y")
    mondays = WeekdayLocator(MONDAY)

    #criando o gráfico
    fig, ax = plt.subplots( figsize=(12,7) )

    df['Adj Close'].plot( grid=True, color='lightgray', linewidth=2)
    df['Band_Sup'].plot(color='yellow', linestyle='--', linewidth=2)
    df['Band_Inf'].plot(color='yellow', linestyle='--', linewidth=2)
    df['MM'].plot(color='#fed976', linestyle='--', linewidth=1)

    plt.title('IBOV - 2020\nBandas de Bollinger - MM: 20p - Bandas: 2dp'
            , fontsize=15
            , color='yellow')
    plt.ylabel('Pontos', fontsize=15, color='lightgray')
    plt.xlabel('Data', fontsize=15, color='lightgray')

    # configurando o grid
    ax.grid(True, color='#555555')
    ax.set_axisbelow(True)
    ax.set_facecolor('black')
    ax.figure.set_facecolor('#121212')
    ax.tick_params(axis='x', colors='white')
    ax.tick_params(axis='y', colors='white')

    # configurando o eixo x
    ax.xaxis.set_major_locator(months)
    ax.xaxis.set_major_formatter(monthsFmt)
    ax.xaxis.set_minor_locator(mondays)
    ax.autoscale_view()

    plt.tight_layout()
    plt.show()
    return None

if __name__ == "__main__":
    dados = cria_bb( "^BVSP", dt.datetime( 2010, 1, 1 ), dt.datetime.now() )

    dados_2020 = dados['2020']
    min = dados_2020[ dados_2020['Adj Close'] == dados_2020['Close'].min ]

    cria_grafico(dados_2020)